In [20]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
#from keras.layers.convolutional import Conv2D
#from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.recurrent import LSTM
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from keras.models import load_model
import pandas as pd
import numpy as np
from keras.layers import Dropout


In [3]:
data = pd.read_csv("data/data/train_data_final_50k.csv", header = None)

In [4]:
train_data = data[data[0]<500]
test_data = data[data[0]>=500]

In [5]:
train_features_timeseries = train_data.iloc[:,22:622]
test_features_timeseries = test_data.iloc[:,22:622]
train_features_18 = train_data[[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]
test_features_18 = test_data[[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

In [6]:
train_labels = train_data.iloc[:,622]
test_labels = test_data.iloc[:,622]

In [7]:
print(train_features_timeseries.shape)
print(test_features_timeseries.shape)
print(train_features_18.shape)
print(test_features_18.shape)
print(train_labels.shape)
print(test_labels.shape)

(42920, 600)
(7080, 600)
(42920, 18)
(7080, 18)
(42920,)
(7080,)


In [8]:
train_features_reshaped = train_features_timeseries.as_matrix().reshape(42920,60,10)
test_features_reshaped = test_features_timeseries.as_matrix().reshape(7080,60,10)

In [9]:
print(train_features_reshaped.shape)
print(test_features_reshaped.shape)

(42920, 60, 10)
(7080, 60, 10)


In [26]:
def get_hot_value(my_list):
    max_val = max(my_list)
    return [int(item == max_val) for item in my_list]

def fit_model(train_features_timeseries,train_features_18, train_labels,test_features_timeseries,test_features_18, test_labels, batch_size, epochs, pickle_name):
    #x_18 = features_18.as_matrix()
    x_18 = Input(shape=(18,))
    #out_18  = Dense(10,activation = 'relu')(x_18)
    
    y_train = np.array(train_labels)
    y_test = np.array(test_labels)
    
    #le = preprocessing.LabelEncoder()
    #y = le.fit_transform(y)

    y_train = y_train.reshape(-1,1)
    y_test = y_test.reshape(-1,1)
    
    ohe = OneHotEncoder(sparse=False)
    y_train = ohe.fit_transform(y_train)
    y_test = ohe.transform(y_test)
    
    #x_lstm = features_timeseries
    x_lstm = Input(shape = (60,10))
    out_lstm = LSTM(60)(x_lstm)
    
    out_lstm_drop = Dropout(0.4)(out_lstm)
    
    merge = concatenate([x_18, out_lstm_drop])
    
    #output_inter1 = Dense(100,activation = 'relu')(merge)
    
    #output_inter2 = Dense(100, activation='relu')(output_inter1)
    
    output_final = Dense(2, activation='softmax')(merge)
    
    model = Model(inputs=[x_18,x_lstm], outputs=[output_final])
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
    print(model.summary())
# plot graph
    #from IPython.display import SVG
    #from keras.utils.vis_utils import model_to_dot
    #SVG(model_to_dot(model).create(prog='dot', format='svg'))
    model.fit([train_features_18,train_features_timeseries],[y_train],epochs=epochs,batch_size=batch_size,validation_data = ([test_features_18,test_features_timeseries],[y_test]))
    model.save(pickle_name)
    return 
    

In [27]:
def test_lstm(test_features_reshaped,test_features_18, test_labels, pickle_name):
    
    model = load_model(pickle_name)
    
    #X = test_features
    y = np.array(test_labels)
    #le = preprocessing.LabelEncoder()
    #y = le.fit_transform(y)

    y = y.reshape(-1,1)
    ohe = OneHotEncoder(sparse=False)
    y = ohe.fit_transform(y)
    
    #Evaluate the model
    scores = model.evaluate([test_features_18,test_features_reshaped], y, verbose=1)
    predicted_labels = model.predict([test_features_18,test_features_reshaped])

    print("Accuracy: ", scores[1]*100)

    hot_list = [get_hot_value(sublist) for sublist in predicted_labels]

    #t  = np.array(y)
    #p  = np.array(hot_list)
    #t = np.argmax(t, axis=1)
    #p = np.argmax(p, axis=1)

    #print("Precision: ", precision_score(t, p, average=None))
    
    return hot_list, ohe

In [28]:
fit_model(train_features_reshaped,train_features_18,train_labels,test_features_reshaped,test_features_18,test_labels,32,10,'multi_input.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 60, 10)       0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 60)           17040       input_8[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 18)           0                                            
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 60)           0           lstm_4[0][0]                     
__________________________________________________________________________________________________
concatenat